In [ ]:
%pip install bitarray

In [2]:
import numpy as np
from scipy.special import softmax
from random import randint

sample_vocab = ['hello', 'world', 'this', 'that', 'apple', 'oranges', 'foo', 'bar', '.']

def sample_from_lm(context=None, seed=0):
    np.random.seed(seed)
    softmax_output = np.random.randint(0, 9, (9))
    return softmax(softmax_output)

for i in range(10):
    print(sample_vocab[np.argmax(sample_from_lm(seed=i))], end=' ')

apple world hello hello that apple bar bar apple this 

In [5]:
from bitarray import bitarray
class Codec():
    def __init__(self, message=None, k=2):
        if isinstance(message, bitarray) or message is None:
            self._message = message
        elif isinstance(message, str):
            self._message = bitarray()
            with open(message, 'rb') as fh:
                self._message.fromfile(fh)
        else:
            ValueError('Expected bitarray or filepath.')
        self._chunk_size = k
        self._message_length = None
        if self._message is not None:
            self._message_length = len(self._message)
    
    def encode(self, out_path):
        """Encode file in text in self._chunk_size'd chunks.
        """
        if self._message is None:
            ValueError('This instance of the codec was not initialized with a message.')
        chunks = [self._message[i:i+self._chunk_size] for i in range(0, self._message_length - self._chunk_size + 1, self._chunk_size)]
        enc_text = sample_vocab[np.argmax(sample_from_lm(seed=0))] + ' '
        for i in range(len(chunks)):
            rankings = np.argsort(sample_from_lm(seed=i+1))
            chunk_to_decimal = int(chunks[i].to01(), base=2)
            enc_text += (sample_vocab[rankings[chunk_to_decimal]] + ' ')
        with open(out_path, 'w') as fh:
            fh.write(enc_text)
        return enc_text

    def decode(self, in_path, out_path, message_length=None):
        """Decode file from text. 
        """
        if message_length is None and self._message_length is None:
            ValueError('Message length not provided.')
        if message_length is not None:
            self._message_length = message_length
        with open(in_path, 'r') as fh:
            enc_text = fh.read()
        bits = bitarray()
        splits = enc_text.split(' ')
        context = splits[0]
        splits = splits[1:-1]
        for i in range(len(splits)):
            rankings = np.argsort(sample_from_lm(seed=i+1)).tolist()
            idx = sample_vocab.index(splits[i])
            chunk = rankings.index(idx)
            bits.extend(bin(chunk)[2:].zfill(self._chunk_size))
        with open(out_path, 'wb') as fh:
            bits.tofile(fh)
        return bits
            

In [6]:
c = Codec('secret_message')
encoded = c.encode('secret.enc')
decoded = c.decode('secret.enc', 'secret.dec', 0)